DFT 离散傅里叶变换
===

DFT，即离散傅里叶变换（Discrete Fourier Transform），是一种将信号从时域（或空间域）转换到频域的数学工具。它在信号处理、图像处理、音频处理、数据压缩和许多其他领域中有着广泛的应用。

### DFT的定义

离散傅里叶变换的公式如下：

$$
X[k] = \sum_{n=0}^{N-1} x[n] \cdot e^{-j \frac{2\pi}{N} kn}
$$

其中：


- **$ X[k] $**：频域信号的第 $ k $ 个分量，表示信号在第 $ k $ 个频率上的幅度和相位。
- **$ x[n] $**：时域信号的第 $ n $ 个采样点，表示信号在第 $ n $ 个时间点的值。
- **$ N $**：信号的总采样点数，表示信号在时域或频域的长度。
- **$ j $**：虚数单位，满足 $ j^2 = -1 $，用于表示复数。
- **$ e $**：自然对数的底数，约等于 2.71828，用于指数函数。
- **$ k $**：频率索引，范围为 $ 0 $ 到 $ N-1 $，表示频域中的位置。
- **$ n $**：时间索引，范围为 $ 0 $ 到 $ N-1 $，表示时域中的位置。

### DFT的关键特性
1. **线性**：DFT是线性的，即多个信号的DFT等于这些信号DFT的和。
2. **周期性**：DFT的结果在频域是周期性的，周期为 $ N $。
3. **对称性**：对于实数信号，DFT的结果是共轭对称的，即 $ X[N-k] = X^*[k] $。
4. **能量守恒**：信号的总能量在时域和频域是相等的。
5. **快速算法**：快速傅里叶变换（FFT）是一种高效的算法，可以快速计算DFT。

### DFT的应用
- **频谱分析**：分析信号的频率成分。
- **滤波**：设计滤波器来去除或增强信号的特定频率成分。
- **图像处理**：在频域中进行图像的滤波、锐化等操作。
- **音频处理**：分析和修改音频信号的频率特性。
- **数据压缩**：通过识别和去除不重要的频率成分来减少数据量。

In [7]:
#r "nuget: OpenCvSharp4.Windows"
using OpenCvSharp;

Installed Packages OpenCvSharp4.Windows, 4.10.0.20240616

In [8]:
var img = Cv2.ImRead("unsplash.jpg", ImreadModes.Grayscale);

// 扩展图像到最佳尺寸
var padded = new Mat();
int m = Cv2.GetOptimalDFTSize(img.Rows);
int n = Cv2.GetOptimalDFTSize(img.Cols); // on the border add zero values
Cv2.CopyMakeBorder(img, padded, 0, m - img.Rows, 0, n - img.Cols, BorderTypes.Constant, Scalar.All(0));

// 添加一个全零的平面
var paddedF32 = new Mat();
padded.ConvertTo(paddedF32, MatType.CV_32F);
Mat[] planes = { paddedF32, Mat.Zeros(padded.Size(), MatType.CV_32F) };
var complex = new Mat();
Cv2.Merge(planes, complex);

// 进行DFT
var dft = new Mat();
Cv2.Dft(complex, dft);

// 计算幅度并转换为对数尺度
// => log(1 + sqrt(Re(DFT(I))^2 + Im(DFT(I))^2))
Cv2.Split(dft, out var dftPlanes);  // planes[0] = Re(DFT(I), planes[1] = Im(DFT(I))

// planes[0] = magnitude
var magnitude = new Mat();
Cv2.Magnitude(dftPlanes[0], dftPlanes[1], magnitude);

Mat magnitude1 = magnitude + Scalar.All(1);  // switch to logarithmic scale
Cv2.Log(magnitude1, magnitude1);

// 如果光谱的行数或列数为奇数，则裁剪光谱
var spectrum = magnitude1[
    new Rect(0, 0, magnitude1.Cols & -2, magnitude1.Rows & -2)];

// 重新排列傅立叶图像的象限，使原点位于图像中心
int cx = spectrum.Cols / 2;
int cy = spectrum.Rows / 2;

var q0 = new Mat(spectrum, new Rect(0, 0, cx, cy));   // Top-Left - Create a ROI per quadrant
var q1 = new Mat(spectrum, new Rect(cx, 0, cx, cy));  // Top-Right
var q2 = new Mat(spectrum, new Rect(0, cy, cx, cy));  // Bottom-Left
var q3 = new Mat(spectrum, new Rect(cx, cy, cx, cy)); // Bottom-Right

// swap quadrants (Top-Left with Bottom-Right)
var tmp = new Mat();
q0.CopyTo(tmp);
q3.CopyTo(q0);
tmp.CopyTo(q3);

// swap quadrant (Top-Right with Bottom-Left)
q1.CopyTo(tmp);
q2.CopyTo(q1);
tmp.CopyTo(q2);

// 将包含浮点值的矩阵转换为
Cv2.Normalize(spectrum, spectrum, 0, 255, NormTypes.MinMax);
spectrum.ConvertTo(spectrum, MatType.CV_8U);

// Show the result
Cv2.ImShow("Input Image", img);
Cv2.ImShow("Spectrum Magnitude", spectrum);

// 计算逆DFT并显示重建图像
var inverseTransform = new Mat();
Cv2.Dft(dft, inverseTransform, DftFlags.Inverse | DftFlags.RealOutput);
Cv2.Normalize(inverseTransform, inverseTransform, 0, 255, NormTypes.MinMax);
inverseTransform.ConvertTo(inverseTransform, MatType.CV_8U);

Cv2.ImShow("Reconstructed by Inverse DFT", inverseTransform);
Cv2.WaitKey();
Cv2.DestroyAllWindows();